### Quick Description

This notebook calculates the toxicity score from each sentence in a subtitles' file. Finally, it persists the data in a DataFrame like output.

In [1]:
import yaml
import glob
import os
import pandas as pd

from tqdm import tqdm
from flair.models import TextClassifier
from flair.data import Sentence

2022-07-07 16:56:17.665037: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-07 16:56:17.675601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-07 16:56:17.675611: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dataset = "coraal"

filepaths = yaml.load(open("/home/guilherme/Desktop/dissertation/config/filepaths.yaml"))

/tmp/ipykernel_512132/670096696.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  filepaths = yaml.load(open("/home/guilherme/Desktop/dissertation/config/filepaths.yaml"))


In [3]:
def calculate_toxicity_scores(save_dir):
    classifier = TextClassifier.load('en-sentiment')
    filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(save_dir, "*"))]

    for filename in tqdm(filenames, total=len(filenames)):
        df_sentences = pd.read_csv(os.path.join(save_dir, filename))
        for i, row in df_sentences.iterrows():
            sentence = Sentence(row.text)
            classifier.predict(sentence)

            df_sentences.loc[i, "flair_score"] = (
                1 - sentence.labels[0]._score
                if sentence.labels[0]._value == "POSITIVE"
                else 1 + sentence.labels[0]._score
            )
            
        # Because 2 is the absolute maximum value
        df_sentences["flair_score"] = (
            (df_sentences["flair_score"] - df_sentences["flair_score"].min())
            / (2 - df_sentences["flair_score"].min())
        )

        df_sentences.to_csv(os.path.join(save_dir, filename), index=False)

In [4]:
calculate_toxicity_scores(filepaths[f"04_{dataset}_scored"])

2022-07-07 16:56:26,247 loading file /home/guilherme/.flair/models/sentiment-en-mix-distillbert_4.pt


100%|███████████████████████████████████████████████████████████████| 142/142 [24:24<00:00, 10.32s/it]
